In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import random

In [3]:
# set device
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# set seed
torch.manual_seed(1)
random.seed(1)
if device == "cuda":
    torch.cuda.manual_seed_all(1)

In [5]:
# Custom Dataset Class to use data_loader(batch_size)
from torch.utils.data import Dataset

class TrainDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.Tensor(x).to(device)
        self.y = torch.LongTensor(y).to(device)
        self.len = len(x)
    def __getitem__(self,index):
        return self.x[index],self.y[index]
    def __len__(self):
        return self.len

In [7]:
# data load
train = pd.read_csv("./data1/mnist_train.csv")
test = pd.read_csv("./data1/mnist_test.csv")
submission = pd.read_csv("./data1/submission.csv")

In [8]:
# check data
print(train.head())
print(test.head())
print(submission.head())

   Unnamed: 0    0    1    2    3    4    5    6    7    8  ...  775  776  \
0           0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1           1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2           2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3           3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4           4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

   777  778  779  780  781  782  783  784  
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  9.0  

[5 rows x 786 columns]
   Unnamed: 0    0    1    2    3    4    5    6    7    8  ...  774  775  \
0           0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
1           1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2           2  0.0

In [9]:
# preprocessing and set x_train, y_train ,x_test
x_train = np.array(train.drop(['0','784'],axis=1))
y_train = np.array(train['784']).astype(int)
x_test = np.array(test.drop(['0'],axis=1))

from sklearn.preprocessing import LabelEncoder,StandardScaler
le = LabelEncoder()
y_train = le.fit_transform(y_train)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# x_train = torch.Tensor(x_train).to(device)
# y_train = torch.LongTensor(y_train).to(device)
x_test = torch.Tensor(x_test).to(device)
# print(x_train.shape)
# print(y_train.shape)
train_dataset= TrainDataset(x_train,y_train)


In [10]:
# set layer
layer1 = torch.nn.Linear(784,512,bias=True).to(device)
layer2 = torch.nn.Linear(512,512,bias=True).to(device)
layer3 = torch.nn.Linear(512,512,bias=True).to(device)
layer4 = torch.nn.Linear(512,512,bias=True).to(device)
layer5 = torch.nn.Linear(512,512,bias=True).to(device)
layer6 = torch.nn.Linear(512,512,bias=True).to(device)
layer7 = torch.nn.Linear(512,len(le.classes_),bias=True).to(device)

relu = torch.nn.ReLU().to(device)
dropout = torch.nn.Dropout(0.3).to(device)

In [11]:
# init layers
torch.nn.init.xavier_normal_(layer1.weight)
torch.nn.init.xavier_normal_(layer2.weight)
torch.nn.init.xavier_normal_(layer3.weight)
torch.nn.init.xavier_normal_(layer4.weight)
torch.nn.init.xavier_normal_(layer5.weight)
torch.nn.init.xavier_normal_(layer6.weight)
torch.nn.init.xavier_normal_(layer7.weight)

Parameter containing:
tensor([[ 0.1136, -0.0801, -0.0845,  ...,  0.0988,  0.0007, -0.0483],
        [ 0.0135, -0.0115,  0.0006,  ..., -0.1359,  0.0526,  0.0597],
        [ 0.0713,  0.0166, -0.0178,  ..., -0.0105, -0.0389,  0.0635],
        ...,
        [-0.0429, -0.0455,  0.0215,  ...,  0.0077, -0.0574, -0.0597],
        [ 0.0161,  0.0767, -0.1157,  ..., -0.0640, -0.0232, -0.0452],
        [ 0.0147,  0.0385,  0.0480,  ...,  0.0763,  0.0079, -0.0304]],
       device='cuda:0', requires_grad=True)

In [12]:
# set model
model = torch.nn.Sequential(layer1,relu,dropout,
                            layer2,relu,dropout,
                            layer3,relu,dropout,
                            layer4,relu,dropout,
                            layer5,relu,dropout,
                            layer6,relu,dropout,
                            layer7).to(device)

In [13]:
# set Learning params and function
# 목표 !! cost 0.04 정도 
# layer 5
# epochs 15 , batch_size 100 -> cost 0.4/// 0.9566
# epochs 50, batch_size 600 -> cost 0.17// 0.9706
# epochs 100, batch_size 600 -> cost 0.073// -> overfitting? 0.9796
# epochs 100, batch_size 1000 -> cost 0.094 // 0.9785
# epochs 100, batch_size 500 -> cost  0.067 // 0.9797
# epochs 100, batch_size 300 -> cost  0.114676796 // 0.9782
# 실습 7번 세팅대로 -> cost 0.24 // 0.96
# layer 5->7, epochs 100, batch_size 500 ->  cost 0.08// 0.9657
# scaling..... -> 0.03


epochs = 150
lr = 0.001
batch_size = 500
optim = torch.optim.Adam(model.parameters(),lr=lr)
loss = torch.nn.CrossEntropyLoss().to(device)
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                         batch_size=batch_size,
                                         shuffle = True,
                                         drop_last=True)

In [14]:
# learning
model.train()
batch_num = len(data_loader)
for epoch in range(epochs+1):
    avg_cost = 0
    for x,y in data_loader:
        optim.zero_grad()
        h = model(x)
        cost = loss(h,y)
        cost.backward()
        optim.step()
        avg_cost += cost

    print("epoch = {:02d} || cost = {:.9f}".format(epoch,avg_cost/batch_num))

epoch = 00 || cost = 0.782132208
epoch = 01 || cost = 0.302209675
epoch = 02 || cost = 0.238780037
epoch = 03 || cost = 0.201770335
epoch = 04 || cost = 0.178874761
epoch = 05 || cost = 0.159889638
epoch = 06 || cost = 0.143201441
epoch = 07 || cost = 0.133738428
epoch = 08 || cost = 0.122382760
epoch = 09 || cost = 0.117153987
epoch = 10 || cost = 0.105922088
epoch = 11 || cost = 0.098460883
epoch = 12 || cost = 0.102145098
epoch = 13 || cost = 0.091569439
epoch = 14 || cost = 0.089838959
epoch = 15 || cost = 0.086246967
epoch = 16 || cost = 0.085018948
epoch = 17 || cost = 0.076771088
epoch = 18 || cost = 0.074758701
epoch = 19 || cost = 0.074379779
epoch = 20 || cost = 0.071793377
epoch = 21 || cost = 0.067144021
epoch = 22 || cost = 0.063606210
epoch = 23 || cost = 0.067003720
epoch = 24 || cost = 0.063963287
epoch = 25 || cost = 0.062828757
epoch = 26 || cost = 0.060665675
epoch = 27 || cost = 0.057294358
epoch = 28 || cost = 0.055789798
epoch = 29 || cost = 0.061102841
epoch = 30

In [15]:
# accuracy 
with torch.no_grad():
    model.eval()
    predict = model(train_dataset.x)
predict = torch.softmax(predict,dim = 1)
predict = torch.argmax(predict,dim = 1)
# print(predict)
# print(dataset.y)
accuracy = (predict == train_dataset.y).float().mean()
print("Accuracy = {:f}".format(accuracy.item()))

Accuracy = 0.999817


In [16]:
# submission
with torch.no_grad():
    model.eval()
    predict = model(x_test)
predict = torch.softmax(predict,dim=1)
predict = torch.argmax(predict,dim=1)
predict = le.inverse_transform(predict.cpu().detach().numpy())
submission['Label'] = predict
print(submission)

        id  Label
0        0      7
1        1      2
2        2      1
3        3      0
4        4      4
...    ...    ...
9995  9995      2
9996  9996      3
9997  9997      4
9998  9998      5
9999  9999      6

[10000 rows x 2 columns]


In [17]:
submission.to_csv("submission.csv",index=False)
# 왜 학습 이후에 계속 한가지 정수로만 결과가 나올까?(batch_size 적용 X)
# batch size가 없으면 한가지 정수로만 예측 결과가 나온다. -> batch size 적용이 필요 
# batch size를 반복문으로 구현하여도 제대로된 예측 결과가 나오지 않음
# Dataloader를 통해 batch size를 사용해야함 
# Data loader 사용을 위해 Dataset Class 의 인스턴스가 필요함 
# CSV 파일에서 읽어온 데이터를 Dataset Class로 만들기 위해 Dataset을 상속받은 클래스 생성
# 생성자, __len__, __getitem__ 함수 구현해야함 


### 위의 추론은 틀렸다. 
# batch_size = 600 기준 , 신경망의 개수가 5개보다 많아지면 Gradient Vanishing이 발생하는 것 같다.
# 6개 부터는 위와 같이 한가지 정수로만 예측결과를 얻게 된다.
# 우선!!제대로된 batch_size의 적용을 위해 Dataset 클래스를 상속하는 커스텀 클래스를 생성해야함!!
# + learning late 조정 와 batch_size 조정 
# ....실습의 데이터 처럼 스케일링 되어서 0~1 사이의 값인 줄 알았는데...아니였다. 스케일링 문제였다.